# **Modelos para detección de transacciones financieras fraudulentas**

## **Descripción**

En este notebook se entrenan y evalúan varios modelos de Machine Learning (Logistic Regression, Random Forest y XGBoost) para la detección de transacciones fraudulentas en tarjetas de crédito. El objetivo es comparar distintas técnicas de clasificación y escoger la mejor en términos de recall, precision y otras métricas adecuadas para un problema altamente desbalanceado.  

Los apartados tratados son:
- Carga del dataset tras un EDA previo.
- División en entrenamiento y test (estratificado).
- Entrenamiento de varios algoritmos.
- Evaluación con métricas (matriz de confusión, AUPRC, etc.).
- Comparación de los resultados.
- Conclusiones finales.

## **Módulos**

In [76]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    confusion_matrix, classification_report, 
    precision_recall_curve, auc, roc_auc_score
)

# Modelos
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

## **Cargo dataset tras EDA**

In [77]:
df = pd.read_csv("creditcard_clean.csv") 
print("Tamaño del dataset:", df.shape)

Tamaño del dataset: (283726, 31)


## **Preparación de features y target**

In [78]:
X = df.drop(columns=['class'])  # El resto de columnas son predictoras
y = df['class']                 # Clase (0: No Fraude, 1: Fraude)

## **División en entrenamiento y test (estratificado por estar desbalanceado)**

In [79]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, stratify=y, random_state=7
)

print("\nDistribución de clases en entrenamiento:")
print(y_train.value_counts(normalize=True))
print("\nDistribución de clases en test:")
print(y_test.value_counts(normalize=True))


Distribución de clases en entrenamiento:
class
0    0.998332
1    0.001668
Name: proportion, dtype: float64

Distribución de clases en test:
class
0    0.998336
1    0.001664
Name: proportion, dtype: float64


## **Función para evaluar los modelos**

In [80]:
def evaluar_modelo(model, X_test, y_test, umbral=0.5):
    """
    Imprime matriz de confusión, metrics, AUPRC, ROC-AUC, etc.
    umbral es el corte para predecir 0 o 1 a partir de la probabilidad
    """
    
    y_proba = model.predict_proba(X_test)[:, 1]  # Probabilidades de la clase positiva (fraude=1)
    y_pred = (y_proba >= umbral).astype(int)     # Predicciones binarias, según umbral
    
    cm = confusion_matrix(y_test, y_pred)        # Matriz de confusión
    
    # Precision, Recall, AUPRC
    precision, recall, thresholds = precision_recall_curve(y_test, y_proba)
    pr_auc = auc(recall, precision)
    
    roc_auc = roc_auc_score(y_test, y_proba)     # ROC AUC
    
    # Clasification Report
    clf_report = classification_report(y_test, y_pred, digits=4)
    
    print("Matriz de Confusión:")
    print(cm)
    print("\nReporte de Clasificación:")
    print(clf_report)
    print(f"AUPRC = {pr_auc:.4f}")
    print(f"ROC-AUC = {roc_auc:.4f}")

## **Entrenamiento de los modelos**

### **Regresión logística**

In [81]:
print("============================================")
print("     MODELO 1: LOGISTIC REGRESSION")
print("============================================")
lr = LogisticRegression(
    solver='lbfgs',
    max_iter=10000,
    random_state=7
)

lr.fit(X_train, y_train)
evaluar_modelo(lr, X_test, y_test, umbral=0.3)

     MODELO 1: LOGISTIC REGRESSION
Matriz de Confusión:
[[70801    13]
 [   35    83]]

Reporte de Clasificación:
              precision    recall  f1-score   support

           0     0.9995    0.9998    0.9997     70814
           1     0.8646    0.7034    0.7757       118

    accuracy                         0.9993     70932
   macro avg     0.9320    0.8516    0.8877     70932
weighted avg     0.9993    0.9993    0.9993     70932

AUPRC = 0.7358
ROC-AUC = 0.9825


### **Random Forest**

In [82]:
from sklearn.model_selection import GridSearchCV
print("\n============================================")
print("     MODELO 2: RANDOM FOREST")
print("============================================")

param_grid = {
    'n_estimators': [25, 50, 100],
    'max_depth': [None],
    'max_features': ['sqrt', 'log2']
}

rf_model = RandomForestClassifier(random_state=7)

grid_search = GridSearchCV(
    estimator=rf_model,
    param_grid=param_grid,
    scoring='average_precision',  # métrica para datos desbalanceados
    cv=3,                         # 3-fold cross-validation
)

grid_search.fit(X_train, y_train)

print("Mejores parámetros:", grid_search.best_params_)
print("Mejor puntuación (average_precision):", grid_search.best_score_)

# Entreno un RandomForest con los mejores parámetros
best_rf = grid_search.best_estimator_

evaluar_modelo(best_rf, X_test, y_test, umbral=0.3)


     MODELO 2: RANDOM FOREST
Mejores parámetros: {'max_depth': None, 'max_features': 'log2', 'n_estimators': 100}
Mejor puntuación (average_precision): 0.8243824897511809
Matriz de Confusión:
[[70802    12]
 [   23    95]]

Reporte de Clasificación:
              precision    recall  f1-score   support

           0     0.9997    0.9998    0.9998     70814
           1     0.8879    0.8051    0.8444       118

    accuracy                         0.9995     70932
   macro avg     0.9438    0.9025    0.9221     70932
weighted avg     0.9995    0.9995    0.9995     70932

AUPRC = 0.8443
ROC-AUC = 0.9432
